In [34]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
from textblob import TextBlob
import nltk
import re

In [8]:
df = pd.read_csv(r'D:\Excel files\Sentiment140 dataset with 1.6 million tweets\training.1600000.processed.noemoticon.csv',encoding='latin1',header=None).sample(8000,random_state=10)

In [9]:
df.head()

,0,1,2,3,4,5
685610,0,2250819430,Sat Jun 20 01:34:59 PDT 2009,NO_QUERY,The_Khristos,"@MrDAprano, honnestly I'm not going to miss th..."
446784,0,2068515884,Sun Jun 07 14:12:53 PDT 2009,NO_QUERY,teale,Tired after spending a restless night with Tie...
1231978,4,1991871464,Mon Jun 01 07:48:38 PDT 2009,NO_QUERY,jenjeahaly,@bradgallaway Yup popiscle Not sure if you've...
1496263,4,2070090087,Sun Jun 07 17:04:40 PDT 2009,NO_QUERY,fashinator1985,@candydiaz LOL save a lil arse for me candy!
944257,4,1795457800,Thu May 14 07:48:29 PDT 2009,NO_QUERY,krrley,@StephanieFizer No problem!! Your stuff is so ...


In [10]:
df = df[[5,0]]
df.columns = ['tweets','sentiment']
df.head()

,tweets,sentiment
685610,"@MrDAprano, honnestly I'm not going to miss th...",0
446784,Tired after spending a restless night with Tie...,0
1231978,@bradgallaway Yup popiscle Not sure if you've...,4
1496263,@candydiaz LOL save a lil arse for me candy!,4
944257,@StephanieFizer No problem!! Your stuff is so ...,4


In [11]:
sent_map = {0:'negative',4:'positive'}

# Feature Engineering

In [12]:
### 1.Word Counts

df['word_counts'] = df['tweets'].apply(lambda x: len(str(x).split()))

In [13]:
df.head()

,tweets,sentiment,word_counts
685610,"@MrDAprano, honnestly I'm not going to miss th...",0,24
446784,Tired after spending a restless night with Tie...,0,22
1231978,@bradgallaway Yup popiscle Not sure if you've...,4,21
1496263,@candydiaz LOL save a lil arse for me candy!,4,9
944257,@StephanieFizer No problem!! Your stuff is so ...,4,19


In [15]:
### 2.Char Counts
df['char_count'] = df['tweets'].apply(lambda x: len(x))

In [16]:
def get_avg_word_len(x):
    words = x.split()
    word_len = 0
    for word in words:
        word_len = word_len+len(word)
    return word_len/len(words)

In [17]:
### 3 . Avg Word Length

df['avg_word_len'] = df['tweets'].apply(lambda x: get_avg_word_len(x))

In [20]:
len("nayan agal")/2


5.0

In [23]:
### 4.Stop Words count
df['stop_words_count'] = df['tweets'].apply(lambda x: len([t for t in x.split() if t in STOP_WORDS]))

In [25]:
### 5.Hashtag and Mentions count

df['hashtag_count'] = df['tweets'].apply(lambda x: len([t for t in x.split() if t.startswith("#")]))
df['mentions_count'] = df['tweets'].apply(lambda x: len([t for t in x.split() if t.startswith("@")]))

In [26]:
### 6. Numeric count

df['numeric_count'] = df['tweets'].apply(lambda x: len([t for t in x.split() if t.isdigit()]))

In [28]:
### 7.Upper Case 

df['uppercase_count'] = df['tweets'].apply(lambda x: len([t for t in x.split() if t.isupper() and len(x)>3]))

# Text Cleaning

In [63]:
##1.
df['tweets'] = df['tweets'].apply(lambda x: x.lower())

In [93]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "i am",
"I've": "i have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that has",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"u":"you",
" ur ":" your ",
" n ":" and ",
'bout':'about',
"cn":"can",
"hve":"have"
}

In [94]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key,value)
        return x
    else:
        return x

In [95]:
##2.
df['tweets'] = df['tweets'].apply(lambda x: cont_to_exp(x))

In [68]:
## 3 To remove accented char
import unicodedata
def remove_accented_char(x):
    x = unicodedata.normalize('NFKD',x).encode('ascii','ignore').decode('utf-8','ignore')
    return x

In [69]:
df['tweets'] = df['tweets'].apply(lambda x: remove_accented_char(x))

In [75]:
## 4. Remove Emails

df['tweets'] = df['tweets'].apply(lambda x: re.sub(r'([a-zA-z0-9+._-]+@[a-zA-z0-9._-]+\.[a-zA-z0-9_-]+)','' , x))

In [77]:
## 5. Punctuation Removal
import string
string.punctuation

df['tweets'] = df['tweets'].apply(lambda x : "".join(i for i in x if i not in string.punctuation))

In [79]:
## 6.Remove RT

df['tweets'] = df['tweets'].apply(lambda x: re.sub('RT',"",x))

In [80]:
## 7. Remove multiple spaces

df['tweets'] = df['tweets'].apply(lambda x : " ".join(x.split()))

In [82]:
## 8.HTML TAGS removal

from bs4 import BeautifulSoup
df['tweets'] = df['tweets'].apply(lambda x : BeautifulSoup(x,'lxml').get_text())

In [83]:
## 9.Remove Stopwords

df['tweets'] = df['tweets'].apply(lambda x : "".join(i for i in x if i not in STOP_WORDS))

In [84]:
## 10.Lemmatize

from textblob import Word
df['tweets'] = df['tweets'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [96]:
# check some processed reviews
import random

i = random.choice(range(len(df)))
print(f"Processed review: \n{df['tweets'].iloc[i]}")  

Processed review: 
drob23 y dd seen ll them but robert


In [97]:
#df.to_csv(r'D:\Excel files\Sentiment140 dataset with 1.6 million tweets\df_preprocessed_12-09-2022.csv')

# Model Building

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler


In [102]:
print(df.shape)
df.head(2)

(8000, 10)


,tweets,sentiment,word_counts,char_count,avg_word_len,stop_words_count,hashtag_count,mentions_count,numeric_count,uppercase_count
685610,mrdprno honnestly m not gong to ms the jntor s...,0,24,128,4.333333,11,0,1,2,0
446784,tred fter spendng restless nght wth ternn off ...,0,22,123,4.545455,10,0,0,0,1


In [131]:
df_feat = df.drop(['tweets','sentiment'],axis=1).reset_index(drop=True)

In [105]:
df_feat.head(2)

,word_counts,char_count,avg_word_len,stop_words_count,hashtag_count,mentions_count,numeric_count,uppercase_count
685610,24,128,4.333333,11,0,1,2,0
446784,22,123,4.545455,10,0,0,0,1


In [106]:
y = df['sentiment']

In [107]:
### 1 . BOW

In [109]:
cv = CountVectorizer()
text_counts = cv.fit_transform(df['tweets'])
df_bow = pd.DataFrame(text_counts.toarray(),columns = cv.get_feature_names())
df_bow.head()

,016,01614948343,06,07,070409,0730,0745,09,0nlhumn,0o,...,zu,zudfunck,zukond,zuljlu,zulkey,zune,zurch,zurdh,zzs,zzzzzs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
### ML Algos

sgd = SGDClassifier(n_jobs=-1,random_state=42,max_iter=200)
lgr = LogisticRegression(random_state=42,max_iter=200)
mnb = MultinomialNB()
rfc = RandomForestClassifier(random_state=42,n_jobs=-1,n_estimators=200)

In [114]:
clf = {'SGD':sgd,'LGR':lgr,'MNB':mnb,'RFC':rfc}

In [124]:
def classify(X,y):
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
    
    for key in clf.keys():
        clf[key].fit(X_train,y_train)
        y_pred  = clf[key].predict(X_test)
        accuracy = accuracy_score(y_test,y_pred)*100
        print(f'The Accuracy of {key} model is {accuracy}%')

In [128]:
##1 BOW
classify(df_bow,y)

The Accuracy of SGD model is 71.5625%
The Accuracy of LGR model is 73.625%
The Accuracy of MNB model is 70.0625%
The Accuracy of RFC model is 74.0%


In [187]:
scaler = MinMaxScaler()
X = scaler.fit_transform(df_bow)
X = pd.DataFrame(X)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
lgr2=LogisticRegression(random_state=42,max_iter=200)    
lgr2.fit(X_train,y_train)
y_pred  = lgr2.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)*100
print(f'The Accuracy of lgr2 model is {accuracy}%')

The Accuracy of lgr2 model is 73.625%


In [129]:
### 2. Manual Features(df_feat)
classify(df_feat,y)

The Accuracy of SGD model is 57.375%
The Accuracy of LGR model is 57.4375%
The Accuracy of MNB model is 58.3125%
The Accuracy of RFC model is 55.50000000000001%


In [132]:
### 3. BOW + Manual Features

X = df_feat.join(df_bow)

In [133]:
classify(X,y)

The Accuracy of SGD model is 70.8125%
The Accuracy of LGR model is 73.3125%
The Accuracy of MNB model is 70.25%
The Accuracy of RFC model is 71.8125%


In [134]:
### 4.TFIDF

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['tweets'])

In [135]:
classify(pd.DataFrame(X.toarray()),y)

The Accuracy of SGD model is 71.75%
The Accuracy of LGR model is 73.9375%
The Accuracy of MNB model is 69.1875%
The Accuracy of RFC model is 73.4375%


In [139]:
### 5. Word2Vec

def get_vec(x):
    doc = nlp(x)
    return doc.vector.reshape(1,-1)

In [138]:
nlp=spacy.load('en_core_web_lg')

In [140]:
df['vec'] = df['tweets'].apply(lambda x: get_vec(x))

In [141]:
X = np.concatenate(df['vec'].to_numpy(),axis=0)

In [143]:
classify(pd.DataFrame(X),y)

The Accuracy of SGD model is 66.8125%
The Accuracy of LGR model is 68.875%
The Accuracy of MNB model is 61.25000000000001%
The Accuracy of RFC model is 64.0%


In [144]:
def pred_w2v(x):
    for key in clf.keys():
        y_pred = clf[key].predict(get_vec(x))
        print(key,"--->",y_pred)

In [145]:
pred_w2v("hi beautiful")

SGD ---> [4]
LGR ---> [4]
MNB ---> [4]
RFC ---> [4]


In [188]:
def predict_cv(text):
    x = cv.transform([text]).toarray() 
    prediction = lgr2.predict(x)
    return prediction

In [189]:
predict_cv('hi beautiful')

array([4], dtype=int64)